In [1]:
import pandas as pd
import pandas_gbq
from google.oauth2 import service_account
import seaborn as sns
import json
import gspread


In [4]:
df = pd.read_excel('D:/Coding/Python/src/DW/sample_-_superstore.xls')


In [ ]:
df


In [9]:
df.head()


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [12]:
df.columns = df.columns.str.replace(' ', '_').str.lower()


In [14]:
df.columns


Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales', 'quantity', 'discount', 'profit'],
      dtype='object')

# Koneksi Python ke Big Query

In [29]:
import os
from google.cloud import bigquery


In [19]:
project_id = 'datawarehouse-457213'
dataset_id = 'superstore'
table_name = 'superstore_data'
key_path   = 'D:/Coding/Python/src/DW/datawarehouse-457213-869c2b633cc3.json'
# Baca credentials dari file JSON
with open(key_path, "r") as file:
    credentials_info = json.load(file)
# Buat credentials object
scopes = ["https://www.googleapis.com/auth/bigquery"]
credentials = service_account.Credentials.from_service_account_info(credentials_info, scopes=scopes)


In [20]:
pandas_gbq.to_gbq(df, f'{dataset_id}.{table_name}', project_id=project_id, if_exists='replace', credentials=credentials)


100%|██████████| 1/1 [00:00<00:00, 14217.98it/s]


In [39]:
query = f'select * from {project_id}.{dataset_id}.{table_name} order by row_id LIMIT 100'
df = pandas_gbq.read_gbq(query, project_id=project_id, credentials=credentials)


Downloading: 100%|██████████|


In [42]:
df.columns


Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'customer_name', 'segment', 'country', 'city', 'state',
       'postal_code', 'region', 'product_id', 'category', 'sub-category',
       'product_name', 'sales', 'quantity', 'discount', 'profit'],
      dtype='object')

In [30]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
client = bigquery.Client()


# ROLL-UP

In [45]:
query = f"""
    SELECT
        region,
        SUM(sales) AS total_sales,
        SUM(profit) AS total_profit
    FROM
        `{project_id}.{dataset_id}.{table_name}`
    GROUP BY
        region
        
    LIMIT 100
"""

df = client.query(query).to_dataframe()
df


C:\Users\PF3DG\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,region,total_sales,total_profit
0,East,678781.2400,91522.7800
1,South,391721.9050,46749.4303
2,Central,501239.8908,39706.3625
3,West,725457.8245,108418.4489


# Drill Down

In [ ]:
query = f"""
    SELECT
        region,
        category,
        sub_category
        SUM(sales) AS total_sales,
        SUM(profit) AS total_profit
    FROM
        `{project_id}.{dataset_id}.{table_name}`
    GROUP BY
        region,
        category
    LIMIT 100
"""

df = client.query(query).to_dataframe()
df 


C:\Users\PF3DG\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,region,category,total_sales,total_profit
0,East,Office Supplies,205516.0550,41014.5791
1,East,Furniture,208291.2040,3046.1658
2,East,Technology,264973.9810,47462.0351
3,South,Office Supplies,125651.3130,19986.3928
4,South,Furniture,117298.6840,6771.2061
5,South,Technology,148771.9080,19991.8314
6,Central,Technology,170416.3120,33697.4320
7,Central,Office Supplies,167026.4150,8879.9799
8,Central,Furniture,163797.1638,-2871.0494
9,West,Office Supplies,220853.2490,52609.8490


# Slice

In [49]:
query = f"""
    SELECT
       *
    FROM
        `{project_id}.{dataset_id}.{table_name}`
    WHERE
         region = 'East'
    LIMIT 50
"""

df = client.query(query).to_dataframe()
df.head()


C:\Users\PF3DG\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,5769,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-LA-10001641,Office Supplies,Labels,Avery 518,3.15,1,0.0,1.5120
1,5770,CA-2015-154900,2015-02-25,2015-03-01,Standard Class,SS-20875,Sung Shariari,Consumer,United States,Leominster,...,1453,East,OFF-PA-10002377,Office Supplies,Paper,Adams Telephone Message Book W/Dividers/Space ...,22.72,4,0.0,10.2240
2,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
3,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
4,8332,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,OFF-ST-10004634,Office Supplies,Storage,"Personal Folder Holder, Ebony",11.21,1,0.0,3.3630


# Dice

In [54]:
query = f"""
    SELECT
       *
    FROM
        `{project_id}.{dataset_id}.{table_name}`
    WHERE
         region IN ('East','South')
         AND category = 'Furniture'
    LIMIT 50
"""

df = client.query(query).to_dataframe()
df.head()


C:\Users\PF3DG\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,9028,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10002597,Furniture,Furnishings,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,3,0.0,6.2244
1,9029,US-2016-152415,2016-09-17,2016-09-22,Standard Class,PO-18865,Patrick O'Donnell,Consumer,United States,Marlborough,...,1752,East,FUR-FU-10004864,Furniture,Furnishings,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,3,0.0,61.3824
2,8333,CA-2016-153269,2016-03-09,2016-03-12,First Class,PS-18760,Pamela Stobb,Consumer,United States,Andover,...,1810,East,FUR-CH-10002647,Furniture,Chairs,"Situations Contoured Folding Chairs, 4/Set",354.90,5,0.0,88.7250
3,6564,CA-2014-166086,2014-05-07,2014-05-12,Standard Class,CT-11995,Carol Triggs,Consumer,United States,Lawrence,...,1841,East,FUR-TA-10003469,Furniture,Tables,Balt Split Level Computer Training Table,194.25,2,0.3,-38.8500
4,6566,CA-2014-166086,2014-05-07,2014-05-12,Standard Class,CT-11995,Carol Triggs,Consumer,United States,Lawrence,...,1841,East,FUR-CH-10004675,Furniture,Chairs,"Lifetime Advantage Folding Chairs, 4/Carton",872.32,4,0.0,244.2496


# Pivot

In [55]:
query = f"""
    SELECT *
    FROM (
        SELECT region, category, sales
        FROM `{project_id}.{dataset_id}.{table_name}`
    )
    PIVOT (
        SUM(sales) FOR category IN ('Furniture', 'Office Supplies', 'Technology')
    )
    LIMIT 100
"""
df = client.query(query).to_dataframe()
df


C:\Users\PF3DG\AppData\Roaming\Python\Python313\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,region,Furniture,Office Supplies,Technology
0,East,208291.2040,205516.055,264973.981
1,South,117298.6840,125651.313,148771.908
2,Central,163797.1638,167026.415,170416.312
3,West,252612.7435,220853.249,251991.832
